In [3]:
%pip install requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import requests
from datetime import datetime
import os

# Ruta de la carpeta y el archivo
folder_path = 'downloads'

# URL del archivo a descargar
url = "https://www.open-epg.com/generate/jmQcr6fwJB.xml"

# Obtener la fecha y hora actual
fechahoraactual = datetime.now().strftime("%Y%m%d%H%M%S")

if not os.path.exists(folder_path):
    os.makedirs(folder_path)
    print(f"Carpeta '{folder_path}' creada.")

# Nombre del archivo con la fecha y hora actual
filename = f"{folder_path}/epg.{fechahoraactual}.xml"

# Descargar el archivo y guardarlo
response = requests.get(url)
with open(filename, 'wb') as file:
    file.write(response.content)

print(f"Archivo guardado como {filename}")

Archivo guardado como downloads/epg.20250101203556.xml
